In [0]:
"""CIFAR 10 Part"""


from __future__ import absolute_import
import numpy as np
import os
import sys
import six
from six.moves import cPickle
from six.moves.urllib.error import HTTPError
from six.moves.urllib.error import URLError
from six.moves.urllib.request import urlopen


def get_file(fname,
             origin,
             untar=False,
             md5_hash=None,
             file_hash=None,
             cache_subdir='datasets',
             hash_algorithm='auto',
             extract=False,
             archive_format='auto',
             cache_dir=None):
    
    if cache_dir is None:
        cache_dir = os.path.expanduser(os.path.join('~', '.keras'))
    if md5_hash is not None and file_hash is None:
        file_hash = md5_hash
        hash_algorithm = 'md5'
    datadir_base = os.path.expanduser(cache_dir)
    if not os.access(datadir_base, os.W_OK):
        datadir_base = os.path.join('/tmp', '.keras')
    datadir = os.path.join(datadir_base, cache_subdir)
    if not os.path.exists(datadir):
        os.makedirs(datadir)

    if untar:
        untar_fpath = os.path.join(datadir, fname)
        fpath = untar_fpath + '.tar.gz'
    else:
        fpath = os.path.join(datadir, fname)

    download = False
    if os.path.exists(fpath):
        # File found; verify integrity if a hash was provided.
        if file_hash is not None:
            if not validate_file(fpath, file_hash, algorithm=hash_algorithm):
                print('A local file was found, but it seems to be '
                      'incomplete or outdated because the ' + hash_algorithm +
                      ' file hash does not match the original value of ' +
                      file_hash + ' so we will re-download the data.')
                download = True
    else:
        download = True

    if download:
        print('Downloading data from', origin)

        class ProgressTracker(object):
            # Maintain progbar for the lifetime of download.
            # This design was chosen for Python 2.7 compatibility.
            progbar = None

        def dl_progress(count, block_size, total_size):
            if ProgressTracker.progbar is None:
                if total_size is -1:
                    total_size = None
                ProgressTracker.progbar = Progbar(total_size)
            else:
                ProgressTracker.progbar.update(count * block_size)

        error_msg = 'URL fetch failure on {}: {} -- {}'
        try:
            try:
                urlretrieve(origin, fpath, dl_progress)
            except URLError as e:
                raise Exception(error_msg.format(origin, e.errno, e.reason))
            except HTTPError as e:
                raise Exception(error_msg.format(origin, e.code, e.msg))
        except (Exception, KeyboardInterrupt) as e:
            if os.path.exists(fpath):
                os.remove(fpath)
            raise
        ProgressTracker.progbar = None

    if untar:
        if not os.path.exists(untar_fpath):
            _extract_archive(fpath, datadir, archive_format='tar')
        return untar_fpath

    if extract:
        _extract_archive(fpath, datadir, archive_format)

    return fpath


def load_batch(fpath, label_key='labels'):
    """Internal utility for parsing CIFAR data.
    # Arguments
        fpath: path the file to parse.
        label_key: key for label data in the retrieve
            dictionary.
    # Returns
        A tuple `(data, labels)`.
    """
    f = open(fpath, 'rb')
    if sys.version_info < (3,):
        d = cPickle.load(f)
    else:
        d = cPickle.load(f, encoding='bytes')
        # decode utf8
        d_decoded = {}
        for k, v in d.items():
            d_decoded[k.decode('utf8')] = v
        d = d_decoded
    f.close()
    data = d['data']
    labels = d[label_key]

    data = data.reshape(data.shape[0], 3, 32, 32)
    return data, labels


def load_data():
    """Loads CIFAR10 dataset.
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    dirname = 'cifar-10-batches-py'
    origin = 'http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
   # path = get_file(dirname, origin=origin, untar=True)

    num_train_samples = 50000

    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    print(x_train.shape, x_test.shape)
    #print(x_train.shape)
    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    return (x_train, y_train), (x_test, y_test)


class Cifar10(object):
  def __init__(self, batch_size=64, one_hot=False, test=False, shuffle=True):
    (x_train, y_train), (x_test, y_test) = load_data()
    if test:
      images = x_test
      labels = y_test
    else:
      images = x_train
      labels = y_train
    if one_hot:
      one_hot_labels = np.zeros((len(labels), 10))
      one_hot_labels[np.arange(len(labels)), labels.flatten()] = 1
      labels = one_hot_labels
    self.shuffle = shuffle
    self._images = images
    self.images = self._images
    self._labels = labels
    self.labels = self._labels
    self.batch_size = batch_size
    self.num_samples = len(self.images)
    if self.shuffle:
      self.shuffle_samples()
    self.next_batch_pointer = 0

  def shuffle_samples(self):
    image_indices = np.random.permutation(np.arange(self.num_samples))
    self.images = self._images[image_indices]
    self.labels = self._labels[image_indices]

  def get_next_batch(self):
    num_samples_left = self.num_samples - self.next_batch_pointer
    if num_samples_left >= self.batch_size:
      x_batch = self.images[self.next_batch_pointer:self.next_batch_pointer + self.batch_size]
      y_batch = self.labels[self.next_batch_pointer:self.next_batch_pointer + self.batch_size]
      self.next_batch_pointer += self.batch_size
    else:
      x_partial_batch_1 = self.images[self.next_batch_pointer:self.num_samples]
      y_partial_batch_1 = self.labels[self.next_batch_pointer:self.num_samples]
      if self.shuffle:
        self.shuffle_samples()
      x_partial_batch_2 = self.images[0:self.batch_size - num_samples_left]
      y_partial_batch_2 = self.labels[0:self.batch_size - num_samples_left]
      x_batch = np.vstack((x_partial_batch_1, x_partial_batch_2))
      y_batch = np.vstack((y_partial_batch_1, y_partial_batch_2))
      self.next_batch_pointer = self.batch_size - num_samples_left
    return x_batch, y_batch


In [0]:
"""Ops part"""


# Reference: https://github.com/openai/improved-gan/blob/master/imagenet/ops.py
import tensorflow as tf
import numpy as np


# TODO: Optionally create utility functions for convolution, fully connected layers here




In [0]:
"""net part"""
def net(input, is_training, dropout_kept_prob):
  # TODO: Write your network architecture here
  # Or you can write it inside train() function
  # Requirements:
  # - At least 5 layers in total
  # - At least 1 fully connected and 1 convolutional layer
  # - At least one maxpool layers
  # - At least one batch norm
  # - At least one skip connection
  # - Use dropout
  raise NotImplementedError

def train():
  # Always use tf.reset_default_graph() to avoid error
  tf.reset_default_graph()
  # TODO: Write your training code here
  # - Create placeholder for inputs, training boolean, dropout keep probablity
  # - Construct your model
  # - Create loss and training op
  # - Run training
  # AS IT WILL TAKE VERY LONG ON CIFAR10 DATASET TO TRAIN
  # YOU SHOULD USE tf.train.Saver() TO SAVE YOUR MODEL AFTER TRAINING
  # AT TEST TIME, LOAD THE MODEL AND RUN TEST ON THE TEST SET
  raise NotImplementedError

def test(cifar10_test_images):
  # Always use tf.reset_default_graph() to avoid error
  tf.reset_default_graph()
  # TODO: Write your testing code here
  # - Create placeholder for inputs, training boolean, dropout keep probablity
  # - Construct your model
  # (Above 2 steps should be the same as in train function)
  # - Create label prediction tensor
  # - Run testing
  # DO NOT RUN TRAINING HERE!
  # LOAD THE MODEL AND RUN TEST ON THE TEST SET
  raise NotImplementedError


In [0]:
import numpy as np
import timeit
from collections import OrderedDict
from pprint import pformat

min_thres=0.60
max_thres=0.70


def compute_score(acc, min_thres, max_thres):
    if acc <= min_thres:
        base_score = 0.0
    elif acc >= max_thres:
        base_score = 100.0
    else:
        base_score = float(acc - min_thres) / (max_thres - min_thres) \
            * 100
    return base_score


if __name__ == '__main__':
    TRAIN = True
    if TRAIN:
        train()
    cifar10_test = Cifar10(test=True, shuffle=False, one_hot=False)
    cifar10_test_images, cifar10_test_labels = cifar10_test._images, cifar10_test._labels

    start = timeit.default_timer()
    np.random.seed(0)
    predicted_cifar10_test_labels = test(cifar10_test_images)
    np.random.seed()
    stop = timeit.default_timer()
    run_time = stop - start
    correct_predict = (cifar10_test_labels.flatten() == predicted_cifar10_test_labels.flatten()).astype(np.int32).sum()
    incorrect_predict = len(cifar10_test_labels) - correct_predict
    accuracy = float(correct_predict) / len(cifar10_test_labels)
    print('Acc: {}. Testing took {}s.'.format(accuracy, stop - start))
    
    score = compute_score(accuracy, min_thres, max_thres)
    result = OrderedDict(correct_predict=correct_predict,
                     accuracy=accuracy, score=score,
                     run_time=run_time)

    with open('result.txt', 'w') as f:
        f.writelines(pformat(result, indent=4))
    print(pformat(result, indent=4))

